In [149]:
import random
import numpy as np
import re
import pandas as pd

In [150]:
# this function takes in[ut a text file and parse it.
# configurationSet is a list of lists in which each element is a list of 3 lists (initial prob, transition matrix, atomic propositions)
# each element in configurationSet corresponds to a configuration.
# configurationSetProb is a list of probabilities of selecting configurations.
def parseInput(path):
    configurationSet=[]
    file=open(path,"r")
    i,j=-1,-1
    for line in file:
        if(i==-1):
            configurationSetProb=formProbList(line)
        
        elif(i%3==0):
            a=formProbList(line)
        elif(i%3==1):
            b=formTranMatrixList(line)
        else:
            c=formApList(line)
        j+=1
        if(j%3==0 and j>0):
            configurationSet.append([a,b,c])
        i+=1
         
    return configurationSet,configurationSetProb

In [151]:
# this function takes a line and separate it according to the given separators.
def my_split(line, seps):
    res = [line]
    for sep in seps:
        line, res = res, []
        for seq in line:
            res += seq.split(sep)
    return res

In [152]:
# its a helper function for parsing input file. this fucntion is used for parsing the transition matrix of a configuration.
def formTranMatrixList(line):
    
    p=my_split(line,seps=["],",'[',']'])
    l=[]
    for i in p:
        if(i=="" or i=='\n'):
            continue
        else:
            a=[]
            for j in i.split(','):
                if not(j==','):
                    a.append(float(j))
            l.append(a)
    return l

In [153]:
# its a helper function for parsing input file. this fucntion is used for parsing the initial prob of a configuration.
def formProbList(line):
    
    a=my_split(line,seps=[',','[',']'])
    l=[]
    for i in range(len(a)-1):
        if i==0:
            continue
        else:
            l.append(float(a[i]))
    return l

In [154]:
# its a helper function for parsing input file. this fucntion is used for parsing the atomic proposition of a configuration.
def formApList(line):
    #print(line)

    #=my_split(line,seps=["],",'[',']'])
    p=my_split(line,seps=["],[","[[","]]"])
    
    #print (p)
    
    l=[]
    
    for i in p:
        if(i=="" or i=='\n' or i=='\t'):
            continue
        else:
            a=[]
            #print (i.split(','))
            for j in i.split(','):
                #print 
                if not(j==','):
                    a.append(int(j))
            l.append(a)
    
    return l

In [155]:
# this function is used for parsing the given query and finding the subformulas in it.
def findindSubformula(s):
   
    openBraceIndex =[]
    subFormula =[]
    openClosePair=[]

    for i in range(len(s)):
        if(s[i]=='('):
            openBraceIndex.append(i)
        elif(s[i]==')'):
            openClosePair.append((openBraceIndex[-1],i))
            openBraceIndex.pop()
            
    for i in openClosePair:
        subFormula.append(s[i[0]:i[1]+1])
    
    return subFormula

In [156]:
def logic(l1,l2,time):

    numStates = len(l1[0][0])
    numConfig = len(l2)
    states=[j for j in range(numStates)]
    configs=[j for j in range(numConfig)]
    
    states = np.array(states)
    configs= np.array(configs)
    
    runApList=[]
    
    currentConfig= np.random.choice(configs, replace=True, p=l2)
    currentState = np.random.choice(states, replace=True, p=l1[currentConfig][0])
    
    # Shall store the sequence of states taken. So, this only has the starting state for now.
    i=0
    stateList = [(currentState,currentConfig,i)]
    runApList.append(l1[currentConfig][2][currentState])
    i+=1
    while(i<time):
        
        nextState = np.random.choice(states,replace=True,p=l1[currentConfig][1][currentState])
        nextConfig=np.random.choice(configs, replace=True, p=l2)
        currentConfig = nextConfig
        currentState = nextState
        stateList.append((currentState,currentConfig,i))
        runApList.append(l1[currentConfig][2][currentState])
        i+=1

    return stateList,runApList

In [206]:
def elementInqueryMaker(query):
    l=[]
    for i in query:
        #print (i)
        if(i=='o'):
            l.append('or')
        elif(i=='r' or i=='n' or i=='d' or i==' '):
            continue
        elif(i=='a'):
            l.append('and')
        elif(i=='U'):
            l.append('U')
        elif(i=='X'):
            l.append('X')
        elif(i=='~'):
            l.append('~')
        else:
            l.append(i)

    return l

In [170]:
def findContentAndClose(elementInQuery):

    openBrace=[]
    closeBrace=[]
    contentAndOpen=[]
    contentAndClose=[]
    for i in range(0,len(elementInQuery)):
        if(elementInQuery[i]=='('):
            openBrace.append(i)
        elif(elementInQuery[i]==')'):
            contentAndOpen[-1].append(i)
            contentAndClose.append(contentAndOpen[-1])
            del contentAndOpen[-1]
            del openBrace[-1]
        else:
            contentAndOpen.append([elementInQuery[i],openBrace[-1],i]) 
            
    return contentAndClose

In [217]:
def fillValidityTable(opr,exp1,exp2,configurationSet,df4,lendingStates):

    if(exp1=='empty' and exp2=='empty'):
        #print ("here")
        checkerAp='('+opr+')'
        for i in lendingStates:
            if int(opr) in configurationSet[i[1]][2][i[0]]:
                df4.loc[[checkerAp],[str(i)]]=True
    elif(opr=='or'):
        checkerAp='('+exp1+'or'+exp2+')'
        for i in lendingStates:
            if(df4.loc[[exp1],[str(i)]].bool() or df4.loc[[exp2],[str(i)]].bool()):
                df4.loc[[checkerAp],[str(i)]]=True
    elif(opr=='and'):
        checkerAp='('+exp1+'and'+exp2+')'
        for i in lendingStates:
            if(df4.loc[[exp1],[str(i)]].bool() and df4.loc[[exp2],[str(i)]].bool()):
                df4.loc[[checkerAp],[str(i)]]=True
    elif(opr=='~'):
        checkerAp='('+exp1+'~'+exp2+')'
        for i in lendingStates:
            if(df4.loc[[exp2],[str(i)]].bool()):
                df4.loc[[checkerAp],[str(i)]]=False
            else:
                df4.loc[[checkerAp],[str(i)]]=True
    elif(opr=='X'):
        checkerAp='('+exp1+'X'+exp2+')'
        for j in range(0,len(lendingStates)):
            if(j==len(lendingStates)-2):
                if(df4.loc[[exp2],[str(lendingStates[j+1])]].bool()):
                    df4.loc[[checkerAp],[str(lendingStates[j])]]=True
                break
            else:
                if(df4.loc[[exp2],[str(lendingStates[j+1])]].bool()):
                    df4.loc[[checkerAp],[str(lendingStates[j])]]=True

            
            if(j==len(lendingStates)-2):
                break;
    elif(opr=='U'):
        checkerAp='('+exp1+'U'+exp2+')'
        flag=False
        check=True
        start=0
        for i in lendingStates:
            if(check==False):
                df4.loc[[checkerAp],[str(i)]]=False
            if(flag==True):
                df4.loc[[checkerAp],[str(i)]]=True
            elif((start==0 and df4.loc[[exp2],[str(i)]].bool()) or (start==0 and df4.loc[[exp2],[str(i)]].bool() and df4.loc[[exp1],[str(i)]].bool())):
                df4.loc[[checkerAp],[str(i)]]=True
                flag=True
                start+=1
            elif(df4.loc[[exp2],[str(i)]].bool() and df4.loc[[exp1],[str(i)]].bool()):
                df4.loc[[checkerAp],[str(i)]]=True
                flag=True
                start+=1
            elif(check and df4.loc[[exp1],[str(i)]].bool() and not(df4.loc[[exp2],[str(i)]].bool())):
                df4.loc[[checkerAp],[str(i)]]=True
                start+=1
            else :
                check=False
               

In [189]:
def findProgRunLists(path,numTimes):

    configurationSet,configurationSetProb=parseInput(path)
    stateList,runApList=logic(configurationSet,configurationSetProb,numTimes)
    
    return configurationSet,configurationSetProb,stateList,runApList

In [190]:
def buildDataframe(query,stateList):
    
    lp=np.unique(findindSubformula(query))
    subF=lp[::-1]
    cols=["subformula"]
    lendingStates =[]
    for i in stateList:
        cols.append(str(i))
        lendingStates.append(i)
        
    df=pd.DataFrame(columns=cols)
    for i in range(len(subF)):
        z=[subF[i]]
        for i in range(len(stateList)):
            z.append(0)
        df2=pd.DataFrame([z],columns=cols)
        df = pd.concat([df,df2], ignore_index=True)
        #df =df.astype(str).astype(int)
    df3 = df.set_index("subformula", drop = False)
    df4=df3.drop(['subformula'], axis=1)

    for i in df4:
        df4[i]=df4[i].astype(str).astype(int).astype(bool)
        
    return df4,lendingStates

In [208]:
def workFlow(query,configurationSet,df4,lendingStates):
    
    elementInQuery=elementInqueryMaker(query)
    contentAndClose=findContentAndClose(elementInQuery)
    for i in contentAndClose:
        if(i[0]=='or' or i[0] =='and' or i[0]=='U' or i[0]=='X' or i[0]=='~'):
            opr=i[0]
            exp1=''
            for j in range(i[1]+1,i[2]):
                exp1+=elementInQuery[j]
            exp2=''
            
            for k in range(i[2]+1,i[3]):
                exp2+=elementInQuery[k]

            fillValidityTable(opr,exp1,exp2,configurationSet,df4,lendingStates)
        else:
            fillValidityTable(i[0],'empty','empty',configurationSet,df4,lendingStates)
    
    return df4
    

In [209]:
def outputAcdToQuery(query):

    df,lendingStates=buildDataframe(query,stateList)
    df2=workFlow(query,configurationSet,df,lendingStates)
    
    return df2,df2.loc[[query],[str(lendingStates[-1])]].bool()

In [226]:
path="/home/shivi/Desktop/fm.txt"
numTimes=10000
configurationSet,configurationSetProb,stateList,runApList=findProgRunLists(path,numTimes)

In [227]:
query="((2)and(~(1)))"
finalDf,outcome=outputAcdToQuery(query)

In [228]:
finalDf

,"(1, 1, 0)","(1, 1, 1)","(0, 0, 2)","(0, 1, 3)","(2, 0, 4)","(2, 0, 5)","(1, 1, 6)","(1, 1, 7)","(1, 1, 8)","(0, 0, 9)",...,"(2, 0, 9990)","(0, 0, 9991)","(2, 1, 9992)","(2, 0, 9993)","(2, 1, 9994)","(2, 1, 9995)","(1, 1, 9996)","(0, 0, 9997)","(2, 0, 9998)","(1, 1, 9999)"
subformula,,,,,,,,,,,,,,,,,,,,,
(~(1)),False,False,False,True,True,True,False,False,False,False,...,True,False,True,True,True,True,False,False,True,False
(2),True,True,False,True,True,True,True,True,True,False,...,True,False,True,True,True,True,True,False,True,True
(1),True,True,True,False,False,False,True,True,True,True,...,False,True,False,False,False,False,True,True,False,True
((2)and(~(1))),False,False,False,True,True,True,False,False,False,False,...,True,False,True,True,True,True,False,False,True,False


In [184]:
runApList

[[1, 2], [1], [2, 3, 5], [2, 3, 5], [2, 3, 5], [1, 2], [1, 2]]

In [168]:
outcome

False